In [1]:
import os

import pandas as pd

In [2]:
SEP       = os.path.sep
ROOT_PATH = SEP.join(os.getcwd().split(SEP)[:-3])
DATA_PATH = f'{ROOT_PATH}/Dataset/books'

In [3]:
books     = pd.read_csv(f'{DATA_PATH}/Books.csv')
users     = pd.read_csv(f'{DATA_PATH}/Users.csv')
ratings   = pd.read_csv(f'{DATA_PATH}/Ratings.csv')

/tmp/ipykernel_2344507/3624816100.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books     = pd.read_csv(f'{DATA_PATH}/Books.csv')


In [4]:
df  = books.merge(ratings, how = 'left', on = 'ISBN')
df  = df.merge(     users, how = 'left', on = 'User-ID')

df_ = df.copy()
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating,Location,Age
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2.0,0.0,"stockton, california, usa",18.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8.0,5.0,"timmins, ontario, canada",NaN
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400.0,0.0,"ottawa, ontario, canada",49.0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676.0,8.0,"n/a, n/a, n/a",NaN
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385.0,0.0,"sudbury, ontario, canada",NaN


In [5]:
df.info()
df.dropna(inplace = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1032345 entries, 0 to 1032344
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   ISBN                 1032345 non-null  object 
 1   Book-Title           1032345 non-null  object 
 2   Book-Author          1032344 non-null  object 
 3   Year-Of-Publication  1032345 non-null  object 
 4   Publisher            1032343 non-null  object 
 5   Image-URL-S          1032345 non-null  object 
 6   Image-URL-M          1032345 non-null  object 
 7   Image-URL-L          1032341 non-null  object 
 8   User-ID              1031136 non-null  float64
 9   Book-Rating          1031136 non-null  float64
 10  Location             1031136 non-null  object 
 11  Age                  753301 non-null   float64
dtypes: float64(3), object(9)
memory usage: 102.4+ MB


In [6]:
df['User-ID']     = df['User-ID'].astype('int')
df['Age']         = df['Age'].astype('int')

## 다 빈치 코드 작가 이름이 2가지로 기재되어 있어 하나로 통일
df['Book-Author'] = df['Book-Author'].astype('string')
df['Book-Author'] = df['Book-Author'].str.replace('DAN BROWN', 'Dan Brown')

In [7]:
df.drop(columns = ['Image-URL-S', 'Image-URL-M'], inplace = True)
df = df[df['Book-Rating'] > 0]
df['Book-Rating'].describe()

count    269617.000000
mean          7.736152
std           1.814537
min           1.000000
25%           7.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: Book-Rating, dtype: float64

In [8]:
df['User-ID'].nunique(), df['Book-Title'].nunique() 

(40543, 109209)

In [9]:
## 유저별로 읽은 책 권수 집계
df.groupby('User-ID')['Book-Title'].agg('count').sort_values()

User-ID
19           1
158169       1
158156       1
158152       1
158121       1
          ... 
16795      959
171118     962
235105    1020
153662    1845
98391     5689
Name: Book-Title, Length: 40543, dtype: int64

In [10]:
book_counts = pd.DataFrame(df['Book-Title'].value_counts())
book_counts.sort_values('Book-Title', ascending = False)

,Book-Title
Wild Animus,473
The Lovely Bones: A Novel,473
The Da Vinci Code,380
The Secret Life of Bees,280
Bridget Jones's Diary,261
...,...
WEEPING ANGEL : WEEPING ANGEL,1
Was die Welt nicht braucht. Von Chatten bis Waschbrettbauch.,1
Knitting Masterpieces,1
I Don't Know What to Say,1


In [11]:
rare_book    = book_counts[book_counts['Book-Title'] <= 100].index
common_book  = df[~df['Book-Title'].isin(rare_book)]
user_book_df = common_book.pivot_table(index = ['User-ID'], columns = ['Book-Title'],
                                       values ='Book-Rating')
user_book_df

Book-Title,1984,1st to Die: A Novel,A Painted House,A Prayer for Owen Meany,A Time to Kill,A Walk to Remember,American Gods,Angels &amp; Demons,Bel Canto: A Novel,Bridget Jones's Diary,...,"The Vampire Lestat (Vampire Chronicles, Book II)",Timeline,To Kill a Mockingbird,"Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson",Watership Down,Where the Heart Is (Oprah's Book Club (Paperback)),White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus
User-ID,,,,,,,,,,,,,,,,,,,,,
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN


In [12]:
book_name = 'Dreamcatcher'
book_name = user_book_df[book_name]
book_name.sort_values(ascending = False)

User-ID
685       10.0
76151     10.0
104278    10.0
109901    10.0
110121    10.0
          ... 
278653     NaN
278683     NaN
278798     NaN
278800     NaN
278843     NaN
Name: Dreamcatcher, Length: 7608, dtype: float64

In [13]:
user_book_df.corrwith(book_name).sort_values(ascending = False).head()

/opt/conda/envs/dove/lib/python3.8/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/conda/envs/dove/lib/python3.8/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


Book-Title
Dreamcatcher                               1.0
Life of Pi                                 1.0
The Girls' Guide to Hunting and Fishing    1.0
Bridget Jones's Diary                      1.0
House of Sand and Fog                      1.0
dtype: float64

In [14]:
rec_book      = user_book_df.corrwith(book_name).sort_values(ascending = False).head()
rec_book_list = list(rec_book.index)
rec_book_list

/opt/conda/envs/dove/lib/python3.8/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/conda/envs/dove/lib/python3.8/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


['Dreamcatcher',
 'Life of Pi',
 "The Girls' Guide to Hunting and Fishing",
 "Bridget Jones's Diary",
 'House of Sand and Fog']

In [15]:
df_author = df[['Book-Title', 'Book-Author']]

df_       = df_author.loc[df_author['Book-Title'].isin(rec_book_list)]
df_       = df_.drop_duplicates(subset = ['Book-Author', 'Book-Author'], keep = 'first')
df_

,Book-Title,Book-Author
12674,Bridget Jones's Diary,Helen Fielding
17936,Life of Pi,Yann Martel
40164,The Girls' Guide to Hunting and Fishing,Melissa Bank
46418,Dreamcatcher,Stephen King
157865,House of Sand and Fog,Andre Dubus III
398924,Dreamcatcher,Dinah McCall
540083,Dreamcatcher,Audrey Osofsky
